To do:
- ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
- Have a look at learning rate and gradient norm clipping which I need to read up on.
    - In addition to this, look at the implications of gradient accumulation steps
    - Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
- Hyperparameter tuning (Alpha, learning rate, batch size so on - not sure how to figure this out)
    - There is precedence for no hyperparameter tuning from the author of the OG NLI model that DEBATE is based on = Due to computational restrains and the points from this paper, no hyperparameter tuning was performed in this case. The model tuning in itself is also not the primary focus in this paper, but simply serves as a tool for the actual inquiry into blame in the Danish Parliament



In [ ]:
%pip install -r "requirements_bert.txt"

In [1]:
import torch
import transformers
import bitsandbytes
import accelerate
import datasets
import numpy as np
import pandas as pd
from peft import LoraConfig, get_peft_model, TaskType
from datetime import datetime
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from sklearn.model_selection import train_test_split
from datasets import Dataset
from keras.losses import binary_crossentropy
from sklearn.metrics import accuracy_score, f1_score, average_precision_score

/work/RuneEgeskovTrust#9638/miniconda3/envs/blame/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-20 22:50:43.562825: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-20 22:50:43.618092: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-20 22:50:44.549074: I tensorflow/core/util/port.cc:153] oneDNN custom 

In [2]:
model_name = "jhu-clsp/mmBERT-base"

quantization_config = BitsAndBytesConfig(
                                        load_in_4bit=True,
                                         bnb_4bit_compute_dtype=torch.bfloat16,
                                         bnb_4bit_quant_type="nf4",
                                         bnb_4bit_use_double_quant=True,
                                         )

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quantization_config,
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at jhu-clsp/mmBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
lora_config = LoraConfig(
    r=8,  # Low-rank dimension
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules="all-linear",  # Fine-tuning the attention layer specifically
)

lora_model = get_peft_model(model, lora_config)
lora_model.print_trainable_parameters()

trainable params: 1,708,048 || all params: 309,239,826 || trainable%: 0.5523


In [4]:
'''
Consider the batch size, could be increased for efficiency purposes.
ADD WEIGHTED LOSS FUNCTION FOR HITS ON 1 FOR POSITIVE LABELLING AS THIS IS WAY MORE RARE = HIGHER "REWARD"
Have a look at learning rate and gradient norm clipping which I need to read up on.
    In addition to this, look at the implications of gradient accumulation steps
    Much of this pipeline was constrained due to computational restrictions which I think was caused by errors and not actual training process.
Early stopping: load_best_model_at_end=True
'''

training_args = TrainingArguments(
    output_dir='./full_results',
    learning_rate=8e-4, # Learning rate copied from mmBERT paper as they found this to perform best
    num_train_epochs=3,
    per_device_train_batch_size=12,
    auto_find_batch_size=True, # Allows for auto adjusting of batch to avoid OOM
    gradient_accumulation_steps=12,  # Simulate larger batch size

    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    fp16=True,  # Enable mixed precision
    dataloader_pin_memory=False,
    remove_unused_columns=True, # Avoiding manual handling of residual text columns
    max_grad_norm=1.0,

    disable_tqdm=False,
)

In [5]:
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True)

In [6]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    #From logits to probabilities
    probs_2d = np.exp(predictions) / np.exp(predictions).sum(axis=1, keepdims=True)
    probs = probs_2d[:, 1]  # positive class extraction
    
    keras_bce = binary_crossentropy(labels, probs)
    keras_bce = float(np.mean(keras_bce.numpy()))  # Converting from keras eagertensor to float value
    
    # Wrapping all metrics to floats for json serialization during model eval
    return {
        'keras_BCE': keras_bce,
        'precision': float(average_precision_score(labels, probs)),
        'accuracy': float(accuracy_score(labels, probs.round())), # Need rounding for these two computations (integer required)
        'f1': float(f1_score(labels, probs.round(), average='macro')) # macro f1 is better for imbalanced dataset
    }

In [7]:

dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/cleaned_training_data_3_4_5_temps.json")


In [8]:
val_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Model_data/validation_set.json")

val_dataframe = val_dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

val_dataset = Dataset.from_pandas(val_dataframe)

tokenized_val = val_dataset.map(tokenize_function)

Map: 100%|██████████| 176/176 [00:00<00:00, 523.16 examples/s] 


In [11]:
test_dataframe = pd.read_json("/work/RuneEgeskovTrust#9638/Bachelor/training_data/subset_cleaned_training_data.json")

test_dataframe = test_dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

test_dataframe = test_dataframe[0:200]

test_dataset = Dataset.from_pandas(test_dataframe)

tokenized_test = test_dataset.map(tokenize_function)

Map: 100%|██████████| 200/200 [00:00<00:00, 546.48 examples/s] 


In [9]:
#val_dataframe = val_dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

dataframe = dataframe[['preceding_sentence', 'text', 'succeeding_sent', 'label']]

#val_dataset = Dataset.from_pandas(val_dataframe)

dataset = Dataset.from_pandas(dataframe)

#tokenized_val = val_dataset.map(tokenize_function)

tokenized_dataset = dataset.map(tokenize_function)

Map: 100%|██████████| 398842/398842 [12:29<00:00, 532.46 examples/s] 


In [13]:
tokenized_dataset.save_to_disk("/work/RuneEgeskovTrust#9638/Bachelor/training_data/tokenized")

Saving the dataset (34/34 shards): 100%|██████████| 398842/398842 [00:13<00:00, 29326.15 examples/s]


In [10]:
trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)


trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [21]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.8130114078521729, 'eval_keras_BCE': 0.8130306005477905, 'eval_precision': 0.6156827579404506, 'eval_accuracy': 0.3806818181818182, 'eval_f1': 0.3487014089288746, 'eval_runtime': 32.2089, 'eval_samples_per_second': 5.464, 'eval_steps_per_second': 0.683, 'epoch': 3.0}


In [ ]:

with open("/work/RuneEgeskovTrust#9638/Bachelor/Bachelor_project/Evalresult5templates.txt", "w") as f:
    f.write(str(eval_results))

In [ ]:
'''
We would expect to see a gradual decrease in both training and validation loss.
If either om them split too far from eachother that indicates issues with the training process.
The process itself should be pretty smooth with no dips either up or down.
'''

In [ ]:
# This is where we should very much remember to save the finetuned model locally as this contains the new weights for use in analyzing new text
lora_model.save_pretrained(f"output/mmBERT/full_model")

In [ ]:
# OG function without bce
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    return {
        'precision': average_precision_score(labels, predictions),
        'accuracy': accuracy_score(labels, predictions),
        'f1': f1_score(labels, predictions, average='macro') # Macro is better suited for imbalanced data
    }